# 📈 NASDAQ Wishlist Tracker - Personal Edition

**For PERSONAL USE ONLY - No sharing, No ngrok needed!**

This notebook runs a web app **inside your browser** using Colab's iframe.

---

## ✨ Features
- ✅ No ngrok required
- ✅ No signup needed
- ✅ Runs entirely in this notebook
- ✅ Beautiful web interface
- ✅ Fully customizable watchlist

---

## 🚀 Quick Start
1. Customize your watchlist below (Cell 2)
2. Run ALL cells (Runtime → Run all)
3. Wait ~30 seconds
4. Web app appears at the bottom!

**That's it!** No tokens, no configuration. 🎉

---
## 📝 Step 1: Customize Your Watchlist

Edit the list below with YOUR favorite NASDAQ stocks:

In [ ]:
# 🎯 YOUR WATCHLIST - Edit this!
MY_WATCHLIST = [
    "AAPL",   # Apple
    "MSFT",   # Microsoft
    "NVDA",   # NVIDIA
    "GOOGL",  # Google
    "AMZN",   # Amazon
    "TSLA",   # Tesla
    "META",   # Meta/Facebook
    "NFLX",   # Netflix
    "AMD",    # AMD
    "INTC",   # Intel
]

print(f"✅ Watchlist configured with {len(MY_WATCHLIST)} stocks")
print(f"Tracking: {', '.join(MY_WATCHLIST)}")

---
## 📦 Step 2: Install Dependencies

Installing required packages...

In [ ]:
%%capture
!pip install flask yfinance pandas pandas-ta flask-cors

---
## 🔧 Step 3: Create Flask App

Building the backend...

In [ ]:
from flask import Flask, render_template_string, jsonify
from flask_cors import CORS
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import threading
import time

app = Flask(__name__)
CORS(app)

# Use the watchlist from above
WATCHLIST = MY_WATCHLIST

def get_company_name(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return info.get('longName', ticker)
    except:
        return ticker

def calculate_indicators(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist_1y = stock.history(period="1y")
        hist_6m = stock.history(period="6mo")
        
        if hist_1y.empty or hist_6m.empty:
            return None
        
        current_price = hist_1y['Close'].iloc[-1]
        week_52_high = hist_1y['High'].max()
        week_52_low = hist_1y['Low'].min()
        week_52_range = f"${week_52_low:.2f} - ${week_52_high:.2f}"
        
        sma_50 = hist_6m['Close'].rolling(window=50).mean().iloc[-1]
        sma_200 = hist_6m['Close'].rolling(window=200).mean().iloc[-1]
        
        rsi_series = ta.rsi(hist_6m['Close'], length=14)
        rsi_14 = rsi_series.iloc[-1] if not rsi_series.empty else None
        
        if pd.notna(sma_50) and pd.notna(sma_200):
            trend = "Bullish" if sma_50 > sma_200 else "Bearish"
        else:
            trend = "N/A"
        
        if pd.notna(rsi_14):
            if rsi_14 < 30:
                signal = "BUY"
            elif rsi_14 > 70:
                signal = "SELL"
            else:
                signal = "HOLD"
        else:
            signal = "N/A"
        
        pct_down = ((week_52_high - current_price) / week_52_high) * 100
        sparkline_data = hist_6m['Close'].tail(30).tolist()
        company_name = get_company_name(ticker)
        
        return {
            'company_name': company_name,
            'ticker': ticker,
            'price': round(current_price, 2),
            'week_52_range': week_52_range,
            'sma_50': round(sma_50, 2) if pd.notna(sma_50) else 'N/A',
            'sma_200': round(sma_200, 2) if pd.notna(sma_200) else 'N/A',
            'rsi_14': round(rsi_14, 2) if pd.notna(rsi_14) else 'N/A',
            'trend': trend,
            'signal': signal,
            'sparkline': sparkline_data,
            'pct_down': round(pct_down, 2)
        }
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        return None

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>My NASDAQ Tracker</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px;
        }
        .container {
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 12px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }
        .header {
            background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }
        .header h1 { font-size: 2em; margin-bottom: 5px; }
        .header p { opacity: 0.9; }
        .controls {
            padding: 20px 30px;
            background: #f8f9fa;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        .refresh-btn {
            background: #28a745;
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 6px;
            cursor: pointer;
            font-size: 16px;
        }
        .refresh-btn:hover { background: #218838; }
        .loading {
            display: none;
            text-align: center;
            padding: 40px;
            font-size: 18px;
        }
        .table-container { overflow-x: auto; padding: 30px; }
        table { width: 100%; border-collapse: collapse; }
        thead { background: #343a40; color: white; }
        th { padding: 15px 10px; text-align: left; font-weight: 600; }
        tbody tr { border-bottom: 1px solid #dee2e6; }
        tbody tr:hover { background: #f8f9fa; }
        td { padding: 15px 10px; }
        .price { font-weight: 700; font-size: 16px; }
        .signal-buy { color: #28a745; font-weight: 700; }
        .signal-sell { color: #dc3545; font-weight: 700; }
        .signal-hold { color: #6c757d; font-weight: 700; }
        .trend-bullish { color: #28a745; font-weight: 600; }
        .trend-bearish { color: #dc3545; font-weight: 600; }
        .sparkline-container { width: 120px; height: 40px; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📈 My NASDAQ Tracker</h1>
            <p>Personal Stock Analysis Dashboard</p>
        </div>
        <div class="controls">
            <div id="lastUpdated">Loading...</div>
            <button class="refresh-btn" onclick="loadData()" id="refreshBtn">🔄 Refresh</button>
        </div>
        <div class="loading" id="loading">Loading market data...</div>
        <div class="table-container">
            <table>
                <thead>
                    <tr>
                        <th>Company</th><th>Ticker</th><th>Price</th><th>52W Range</th>
                        <th>SMA 50</th><th>SMA 200</th><th>RSI</th><th>Trend</th>
                        <th>Signal</th><th>Sparkline</th><th>% Down</th>
                    </tr>
                </thead>
                <tbody id="tableBody"></tbody>
            </table>
        </div>
    </div>
    <script>
        const charts = {};
        function createSparkline(id, data) {
            const ctx = document.getElementById(id).getContext('2d');
            if (charts[id]) charts[id].destroy();
            charts[id] = new Chart(ctx, {
                type: 'line',
                data: {
                    labels: data.map((_, i) => i),
                    datasets: [{
                        data: data,
                        borderColor: '#667eea',
                        borderWidth: 2,
                        fill: false,
                        pointRadius: 0,
                        tension: 0.4
                    }]
                },
                options: {
                    responsive: true,
                    maintainAspectRatio: false,
                    plugins: { legend: { display: false }, tooltip: { enabled: false } },
                    scales: { x: { display: false }, y: { display: false } }
                }
            });
        }
        async function loadData() {
            const loading = document.getElementById('loading');
            const tableBody = document.getElementById('tableBody');
            const refreshBtn = document.getElementById('refreshBtn');
            loading.style.display = 'block';
            refreshBtn.disabled = true;
            try {
                const response = await fetch('/api/watchlist');
                const data = await response.json();
                tableBody.innerHTML = '';
                data.forEach((stock, index) => {
                    const sparklineId = `spark-${index}`;
                    const row = `<tr>
                        <td>${stock.company_name}</td>
                        <td><strong>${stock.ticker}</strong></td>
                        <td class="price">$${stock.price}</td>
                        <td>${stock.week_52_range}</td>
                        <td>${stock.sma_50}</td>
                        <td>${stock.sma_200}</td>
                        <td>${stock.rsi_14}</td>
                        <td class="trend-${stock.trend.toLowerCase()}">${stock.trend}</td>
                        <td class="signal-${stock.signal.toLowerCase()}">${stock.signal}</td>
                        <td><div class="sparkline-container"><canvas id="${sparklineId}"></canvas></div></td>
                        <td>${stock.pct_down}%</td>
                    </tr>`;
                    tableBody.innerHTML += row;
                    setTimeout(() => createSparkline(sparklineId, stock.sparkline), 100);
                });
                document.getElementById('lastUpdated').textContent = `Updated: ${new Date().toLocaleString()}`;
            } catch (error) {
                tableBody.innerHTML = `<tr><td colspan="11" style="color:red;">Error: ${error.message}</td></tr>`;
            } finally {
                loading.style.display = 'none';
                refreshBtn.disabled = false;
            }
        }
        window.onload = loadData;
    </script>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/watchlist')
def get_watchlist_data():
    results = []
    for ticker in WATCHLIST:
        print(f"Processing {ticker}...")
        data = calculate_indicators(ticker)
        if data:
            results.append(data)
    return jsonify(results)

print("✅ Flask app created successfully!")

---
## 🚀 Step 4: Launch Your Personal Web App

**This will display the app right here in the notebook!**

Run this cell and scroll down to see your dashboard.

In [ ]:
import threading
from google.colab import output

# Start Flask in background thread
def run_app():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app, daemon=True)
thread.start()

# Wait for Flask to start
import time
time.sleep(3)

print("\n" + "="*60)
print("🎉 YOUR WEB APP IS RUNNING!")
print("="*60)
print("\n📱 The dashboard will appear below in a few seconds...")
print("\n💡 Tips:")
print("   - Click 'Refresh' to update prices")
print("   - Keep this cell running (don't stop it!)")
print("   - Edit watchlist in Cell 2 and re-run to change stocks")
print("\n" + "="*60 + "\n")

# Display the app in an iframe
output.serve_kernel_port_as_iframe(5000, height=800)

---

## 🎯 How to Use

### Refresh Data
Click the **🔄 Refresh** button in the dashboard above

### Change Watchlist
1. Go back to **Cell 2**
2. Edit the `MY_WATCHLIST` array
3. **Runtime** → **Run all** (or Ctrl+F9)

### Understanding Signals

#### Signal Colors:
- 🟢 **BUY** - RSI < 30 (Stock oversold, potential buy)
- 🔴 **SELL** - RSI > 70 (Stock overbought, potential sell)
- ⚪ **HOLD** - RSI 30-70 (Neutral, hold position)

#### Trend Colors:
- 🟢 **Bullish** - SMA50 > SMA200 (Upward trend)
- 🔴 **Bearish** - SMA50 < SMA200 (Downward trend)

### Export Data
Run the cell below to download as CSV:


In [ ]:
# Optional: Export current data to CSV
import pandas as pd
from google.colab import files
from datetime import datetime

def export_to_csv():
    results = []
    for ticker in MY_WATCHLIST:
        data = calculate_indicators(ticker)
        if data:
            results.append(data)
    
    df = pd.DataFrame(results)
    filename = f"nasdaq_tracker_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(filename, index=False)
    
    print(f"✅ Exported to {filename}")
    files.download(filename)

# Uncomment and run to export:
# export_to_csv()

---

## 💡 Customization Ideas

### Add More Stocks
```python
MY_WATCHLIST = [
    "AAPL", "MSFT", "NVDA",
    "GOOGL", "AMZN", "TSLA",
    # Add up to 50 stocks!
]
```

### Focus on Specific Sectors
```python
# Tech stocks
MY_WATCHLIST = ["AAPL", "MSFT", "NVDA", "AMD", "INTC"]

# EVs
MY_WATCHLIST = ["TSLA", "RIVN", "LCID", "NIO"]

# Streaming
MY_WATCHLIST = ["NFLX", "DIS", "PARA", "WBD"]
```

### Schedule Auto-Refresh
Add this to auto-refresh every 5 minutes:
```python
import time
while True:
    time.sleep(300)  # 5 minutes
    print("Auto-refreshing...")
```

---

## ⚠️ Important Notes

1. **Keep the cell running** - Don't stop Cell 5 or the app will stop
2. **Colab timeout** - Free Colab sessions last ~12 hours
3. **No sharing** - This works ONLY in YOUR notebook
4. **Rate limits** - Yahoo Finance has usage limits
5. **Save your work** - File → Save a copy in Drive

---

## 🎓 What You're Learning

By using this tracker, you're learning:
- Technical analysis indicators (SMA, RSI)
- Market trends and momentum
- Data-driven investment decisions
- Python for finance

---

## ⚖️ Disclaimer

**This is for educational purposes only. NOT financial advice!**

Always do your own research and consult with a financial advisor before making investment decisions.

---

## 📚 Resources

- [Understanding SMA](https://www.investopedia.com/terms/s/sma.asp)
- [RSI Explained](https://www.investopedia.com/terms/r/rsi.asp)
- [Technical Analysis Basics](https://www.investopedia.com/technical-analysis-4689657)

---

**Made with ❤️ for personal stock tracking**